In [74]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg
from WhatIfAnalysis import GoalSeek
# import sectionproperties as shape

from ASCE import load_combos
import sys
sys.path.append('/Users/davidhansonc/Desktop') #add desktop to PATH for development modules
from WLS.WindParameters import WindParameters #Wind Loads Module

from steel_design.AISC import aisc
steel_density = 490 / 12**3 #pci

from concrete_design import material_properties as conc
from concrete_design.steel_reinforcement import rebar_df as rebar
conc_density = 150.0 #pcf

from masonry_design import material_properties as cmu
from masonry_design.material_properties import NW_CMU #psf

from wood_design.sawn_lumber.ref_design_values.table_4a import table_4a_load
from wood_design.sawn_lumber.ref_design_values.adjustment_factors import SawnLumberAdjustmentFactors
from wood_design.sawn_lumber.ref_design_values.C_F import C_F_load
table_4a = table_4a_load(species="DF")


<div style="page-break-after: always;"></div>

# Calculations

In [75]:
aisc

,Type,T_F,W,A,d,ddet,Ht,h,OD,bf,...,rts,ho,PA,PA2,PB,PC,PD,T,WGi,WGo
Shape,,,,,,,,,,,,,,,,,,,,,
W44x335,W,F,335.00,98.50,44.0,44.000,NaN,NaN,NaN,15.9,...,4.24,42.2,132.0,NaN,148.0,104.0,120.0,38.0,5.5,NaN
W44x290,W,F,290.00,85.40,43.6,43.625,NaN,NaN,NaN,15.8,...,4.20,42.0,131.0,NaN,147.0,103.0,119.0,38.0,5.5,NaN
W44x262,W,F,262.00,77.20,43.3,43.250,NaN,NaN,NaN,15.8,...,4.17,41.9,131.0,NaN,147.0,102.0,118.0,38.0,5.5,NaN
W44x230,W,F,230.00,67.80,42.9,42.875,NaN,NaN,NaN,15.8,...,4.13,41.7,130.0,NaN,146.0,102.0,118.0,38.0,5.5,NaN
W40x655,W,T,655.00,193.00,43.6,43.625,NaN,NaN,NaN,16.9,...,4.71,40.1,132.0,NaN,149.0,104.0,121.0,34.0,7.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pipe5xxS,PIPE,F,38.60,10.70,NaN,NaN,NaN,NaN,5.563,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pipe4xxS,PIPE,F,27.60,7.66,NaN,NaN,NaN,NaN,4.500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pipe3xxS,PIPE,F,18.60,5.17,NaN,NaN,NaN,NaN,3.500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
aisc[aisc['Type'] == 'PIPE']

,Type,T_F,W,A,d,ddet,Ht,h,OD,bf,...,rts,ho,PA,PA2,PB,PC,PD,T,WGi,WGo
Shape,,,,,,,,,,,,,,,,,,,,,
Pipe26STD,PIPE,F,103.00,28.200,NaN,NaN,NaN,NaN,26.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pipe24STD,PIPE,F,94.70,26.000,NaN,NaN,NaN,NaN,24.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pipe20STD,PIPE,F,78.70,21.600,NaN,NaN,NaN,NaN,20.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pipe18STD,PIPE,F,70.70,19.400,NaN,NaN,NaN,NaN,18.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pipe16STD,PIPE,F,62.60,17.200,NaN,NaN,NaN,NaN,16.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pipe14STD,PIPE,F,54.60,15.000,NaN,NaN,NaN,NaN,14.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pipe12STD,PIPE,F,49.60,13.700,NaN,NaN,NaN,NaN,12.750,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pipe10STD,PIPE,F,40.50,11.500,NaN,NaN,NaN,NaN,10.750,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pipe8STD,PIPE,F,28.60,7.850,NaN,NaN,NaN,NaN,8.625,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
w = aisc.loc["Pipe6STD", "W"] #lb/ft

H = 60 #ft
W = w * H #lbs
W

1140.0

In [78]:
V = 105 #mph
exposure_Cat = "C"
Height = [15, 20, 25, 30, 40, 50, 60]
for h in Height:
	Wind_data = WindParameters(V, exposure_Cat, h)
	print(f"q_z_{h} = {round(Wind_data.q_z, 2)} psf")


q_z_15 = 20.37 psf
q_z_20 = 21.64 psf
q_z_25 = 22.68 psf
q_z_30 = 23.56 psf
q_z_40 = 25.04 psf
q_z_50 = 26.24 psf
q_z_60 = 27.27 psf


In [79]:

D = 6 / 12 #ft (width of pole)
G = 0.85
h = 60
q_z = WindParameters(V, exposure_Cat, h).q_z
print(f"q_z = {round(q_z, 2)} psf")

h/D, D * sqrt(q_z)

q_z = 27.27 psf


(120.0, 2.6108847269373925)

In [80]:
def calculate_C_f(D, q_z, h):
    h_over_D = h / D
    if D * sqrt(q_z) > 2.5:
        # Define the points for interpolation
        x_points = [1, 7, 25]
        y_points = [0.5, 0.6, 0.7]
    else:
        # Define the points for interpolation
        x_points = [1, 7, 25]
        y_points = [0.7, 0.8, 1.2]
    
    # Interpolate C_f based on h_over_D
    C_f = np.interp(h_over_D, x_points, y_points)
    
    return C_f

In [82]:
C_f = calculate_C_f(D, q_z, h)
print(C_f)

F = q_z * G * C_f * D #plf
f"F/ft = {round(F, 2)} plf"

0.7


'F/ft = 8.11 plf'